 # ISS Position Analysis

## Objectives:

* [ ] Convert the ISS position at a specific date to Azimuth and Elevation angles.
    * [X] Use SNC Rocky-Mountain Campus as reference.
    * [ ] Use Astropy and other APIs to build solution.
    * [ ] Use internally-derived equations.
    * [ ] Verify these angles make sense.

* [ ] Convert Az/El angles back to Equitorial Coordinates

* [ ] Verify logic using TLE data of ISS/Zarya.

## Import Requires Python Libraries

In [1]:
from datetime import datetime, timedelta
import logging
import math

import pytz
from pytz import timezone

from astropy import units as u
from astropy.coordinates import AltAz, CartesianDifferential, CartesianRepresentation, EarthLocation, ICRS, ITRS, SkyCoord, TEME, get_body, get_sun
from astropy.time import Time
from astropy.visualization import quantity_support

import numpy as np

from sgp4.api import Satrec

Configure logger.
* **Note:** Use `logging.INFO` unless troubleshooting issues.  Then use `logging.DEBUG`.

In [2]:
logging.basicConfig( level = logging.DEBUG )
logger = logging.getLogger('main')

## Test Configuration

This test is designed around the following information:

* RMS Facility Location (Lon/Lat/Alt):  `-104.844^{\circ}`, `39.544^{\circ}`, `1970 m`
* Time of Event:
    * JD: `2460878.45347`
    * MJD: `60877.95347`
    * Greg (MDT): `2025-07-21 15:53:00 UTC-7`
    * Greg (UTC): `2025-07-21 22:53:00`
* ISS Position:
    * J2000 Eq (Ra/Dec): $(9h, 25m, 25.80s, \small{+} 40^{\circ}, 40', 46.0'')$
    * J2000 Eq (Ra/Dec): $(9h, 27m, 03.31s, \small{+} 40^{\circ}, 40', 17.0'')$
    * Ground Position (LLA): $35.91^{\circ}, -104.8^{\circ}, 419 \textit{km}$
* Expected Az/El Angles:
    * Az: $284^{\circ}, 15' 17.0''$, El: $62^{\circ}, 11', 21.5''$ 


### Expected Results
![Stellarium Results](./images/ISS_Stellarium_20250720.png)

Here is the expected per Wolfram Alpha
* Input: `Position of the ISS at 22:53:00 UTC on July 21, 2025`
![Wolfram Alpha](./images/wolfram_20250720.png)

### Right Ascension

In [3]:
ra_hms_date  = ( 9, 25, 25.8  )
ra_hms_j2000 = ( 9, 27,  3.31 )

### Declination

In [4]:
dec_dms_date  = ( 40, 46, 53.9 )
dec_dms_j2000 = ( 40, 40, 17.0 )

### Event Date

In [5]:
dt = datetime( year   = 2025,
               month  = 7,
               day    = 21,
               hour   = 22,
               minute = 53,
               second = 00 )
dt_utc = timezone('UTC').localize( dt )
logger.info( f'UTC Date: {dt_utc.strftime("%x %X")}' )

dt_local = dt_utc.astimezone(pytz.timezone('America/Denver'))
logger.info( f'Denver Date: {dt_local.strftime("%x %X")}' )

event_jd_exp = 2460878.45347
event_mjd_exp  = 60877.95347

INFO:main:UTC Date: 07/21/25 22:53:00
INFO:main:Denver Date: 07/21/25 16:53:00


### Observer Position (Geographic)

In [6]:
obs_pos_lla = [ -104.844, 39.544, 1790 ]

### Two-Line-Element (TLE) of the ISS
Retrieved July 21, 2025 from [Celestrak](https://celestrak.org/NORAD/elements/gp.php?GROUP=stations&FORMAT=tle).

```
ISS (ZARYA)             
1 25544U 98067A   25202.63188906  .00007327  00000+0  13628-3 0  9993
2 25544  51.6344 136.3369 0002363 106.2254 313.1028 15.49989292520508
```

In [7]:
iss_tle_str = [ 'ISS (ZARYA)',
                '1 25544U 98067A   25202.63188906  .00007327  00000+0  13628-3 0  9993',
                '2 25544  51.6344 136.3369 0002363 106.2254 313.1028 15.49989292520508' ]

## Supporting Functions

### Converting Hour/Minute/Seconds to Decimal Hours

$$
h = \textit{hr} + \frac{\textit{min}}{60} + \frac{\textit{sec}}{360}
$$

In [8]:
def hms_to_hours( hms: tuple ):
    return hms[0] + hms[1]/60. + hms[2]/360.

### Converting Deg/Min/Sec to Decimal Degrees

$$
\textit{deg} = \frac{\textit{hr}}{24} \cdot 360
$$

In [9]:
def hours_to_dms( hr ):
    return (hr / 24.) * 360.

### Converting Geographic to ECEF

In [10]:
def lla_to_ecef( pos_lla ):

    lon_deg = pos_lla[0]
    lat_deg = pos_lla[1]
    alt_m   = pos_lla[2]

    a = 6378137
    a_sq = a**2
    e = 8.181919084261345e-2
    e_sq = e**2
    b_sq = a_sq*(1 - e_sq)

    lat_rad = math.radians( lat_deg )
    lon_rad = math.radians( lon_deg )

    N = a / math.sqrt( 1. - e_sq * math.sin( lat_rad ) ** 2 )
    x = ( N + alt_m ) * math.cos( lat_rad ) * math.cos( lon_rad )
    y = ( N + alt_m ) * math.cos( lat_rad ) * math.sin( lon_rad )
    z = ( ( b_sq / a_sq ) * N + alt_m ) * math.sin( lat_rad )

    return np.array( [x, y, z], dtype = np.float64 )

### Converting ECEF to Geographic

In [11]:
def ecef_to_lla( pos_ecef ):
	
    # x, y and z are scalars or vectors in meters
	x = pos_ecef.flatten()[0]
	y = pos_ecef.flatten()[1]
	z = pos_ecef.flatten()[2]

	a=6378137
	a_sq=a**2
	e = 8.181919084261345e-2
	e_sq = 6.69437999014e-3

	f = 1/298.257223563
	b = a*(1-f)

	# calculations:
	r = math.sqrt(x**2 + y**2)
	ep_sq  = (a**2-b**2)/b**2
	ee = (a**2-b**2)
	f = (54*b**2)*(z**2)
	g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
	c = (e_sq**2)*f*r**2/(g**3)
	s = (1 + c + math.sqrt(c**2 + 2*c))**(1/3.)
	p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
	q = math.sqrt(1 + 2*p*e_sq**2)
	r_0 = -(p*e_sq*r)/(1+q) + math.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
	u = math.sqrt((r - e_sq*r_0)**2 + z**2)
	v = math.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
	z_0 = (b**2)*z/(a*v)
	h = u*(1 - b**2/(a*v))
	phi = math.atan((z + ep_sq*z_0)/r)
	lambd = math.atan2(y, x)


	return phi*180/math.pi, lambd*180/math.pi, h

### Create ECEF to NED Rotation Matrix


In [12]:
def ecef_to_ned_matrix( lla_pos ):

    lon_ref = math.radians( lla_pos[0] )
    lat_ref = math.radians( lla_pos[1] )

    M = np.eye( 3 )
    M[0,0]=-math.sin(lat_ref)*math.cos(lon_ref)
    M[0,1]=-math.sin(lat_ref)*math.sin(lon_ref)
    M[0,2]= math.cos(lat_ref)
	
    M[1,0]=-math.sin(lon_ref)
    M[1,1]= math.cos(lon_ref)
    M[1,2]= 0

    M[2,0]=-math.cos(lat_ref)*math.cos(lon_ref)
    M[2,1]=-math.cos(lat_ref)*math.sin(lon_ref)
    M[2,2]=-math.sin(lat_ref)

    return M

## NED to Topocentric


In [13]:
def norm_degree( deg, min_val = 0, max_val = 360 ):
    range = max_val - min_val
    while deg > max_val:
        deg -= range
    while deg < min_val:
        deg += range
    return deg

In [14]:
def ned_to_horizon( ned ):

    ned_flat = ned.flatten()
    az = norm_degree( math.degrees( math.atan2( ned_flat[1], ned_flat[0] ) ) )
    el = norm_degree( math.degrees( -math.asin( ned_flat[2] ) ), -90, 90 )

    return (az, el)

Convert the Sensor Position to ECEF

## Validating Right Ascension

For this secion, we test conversions between different representations of Right Ascension. 

***Right Ascension:***

Represented in Hours, Minutes, and Seconds

In [15]:
ra_deg_date  = hours_to_dms( hms_to_hours( ra_hms_date ) )
ra_deg_j2000 = hours_to_dms( hms_to_hours( ra_hms_j2000 ) )

In [16]:
logger.info( f'Right Ascension (Date) : {ra_deg_date:.6f}' )
logger.info( f'Right Ascension (J2000): {ra_deg_j2000:.6f}' )

INFO:main:Right Ascension (Date) : 142.325000
INFO:main:Right Ascension (J2000): 141.887917


***Declination:***
 Input is often represented in Degrees, Minutes, Seconds (DMS).  We need to convert to Decimal-Degrees.

In [17]:
de_deg_date  = hms_to_hours( dec_dms_date )
de_deg_j2000 = hms_to_hours( dec_dms_j2000 )

In [18]:
logger.info( f'Declination (Date) : {de_deg_date:.6f}' )
logger.info( f'Declination (J2000): {de_deg_j2000:.6f}' )

INFO:main:Declination (Date) : 40.916389
INFO:main:Declination (J2000): 40.713889


## Create Astropy Objects

### Expected ISS Positions

In [19]:
iss_ref_pos_date = SkyCoord( ra    = ra_deg_date,
                             dec   = de_deg_date,
                             unit  = (u.hourangle, u.deg),
                             frame = ICRS )

iss_ref_pos_j2000 = SkyCoord( ra    = ra_deg_j2000,
                              dec   = de_deg_j2000,
                              unit  = (u.hourangle, u.deg),
                              frame = ICRS )

### Position of Sensor at RMC

In [20]:
obs_gnd_pos = EarthLocation.from_geodetic( lon    = obs_pos_lla[0] * u.deg,
                                           lat    = obs_pos_lla[1] * u.deg,
                                           height = obs_pos_lla[2] * u.m )

### Event Time

In [21]:
sensor_time = Time( event_jd_exp, format = 'jd' )#Time( dt_local )

event_mjd_act = sensor_time.to_value("mjd")
event_jd_act  = sensor_time.to_value("jd")

mjd_error = abs(event_mjd_act - event_mjd_exp)
jd_error  = abs(event_jd_act  - event_jd_exp)

logger.info( f'  Event Date: {sensor_time}' )
logger.info( f'         MJD: {event_mjd_act}, Error: {mjd_error:.2f}' )
logger.info( f'          JD: {event_jd_act}, Error: {jd_error:.2f}' )

INFO:main:  Event Date: 2460878.45347
INFO:main:         MJD: 60877.953470000066, Error: 0.00
INFO:main:          JD: 2460878.45347, Error: 0.00


### Observer's Frame-of-Reference

In [22]:
frame = AltAz( obstime  = sensor_time,
               location = obs_gnd_pos )
logger.info( frame )

INFO:main:<AltAz Frame (obstime=2460878.45347, location=(-1262105.1446924414, -4762065.809473791, 4040211.4453365966) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron)>


### Solve for the Az/El Angle for Frame-of-Reference, at Specific Time

<span style="color:red"><b>Note: This code doesn't work yet.  There seems to be conversion errors with the time zone.</b></span>

In [23]:
alt_az_date = iss_ref_pos_date.transform_to( frame )
logger.info( alt_az_date )

INFO:main:<SkyCoord (AltAz: obstime=2460878.45347, location=(-1262105.1446924414, -4762065.809473791, 4040211.4453365966) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (17.41119886, -6.68945828)>


## Solving Azimuth and Elevation using ECEF Positions

As a sanity check, we can convert the ISS position to ECEF, then estimate the vector's Az/El angles using the Obs -> ISS vector.  This will involve a conversion to NED, then some light trig.

### Using sgp4 for processing ISS TLE Info

In [24]:
obs_pos_ecef = lla_to_ecef( obs_pos_lla )
logger.info( f'Observer Position ECEF: {obs_pos_ecef}' )

INFO:main:Observer Position ECEF: [-1262105.14469244 -4762065.80947379  4040211.4453366 ]


In [25]:
iss_sat_info = Satrec.twoline2rv( iss_tle_str[1], iss_tle_str[2] )
logger.debug(iss_sat_info)

e, r, v = iss_sat_info.sgp4( event_jd_act, 0. )

iss_tle_teme_p = np.array( list(r) )
iss_tle_teme_v = np.array( list(v) )
logger.debug( f'\n  - error code: {e}\n  - relative_position (km): {iss_tle_teme_p}\n  - velocity (km/s): {iss_tle_teme_v}\n' )

time = Time( event_jd_act, format='jd' )
iss_teme_p = CartesianRepresentation( iss_tle_teme_p * u.km )
iss_teme_v = CartesianDifferential( iss_tle_teme_v * u.km / u.s )

iss_teme   = TEME( iss_teme_p.with_differentials( iss_teme_v ), obstime=time )
itrs = iss_teme.transform_to( ITRS( obstime = time ) )

iss_itrs_p = itrs.cartesian.xyz.value * 1000.
iss_itrs_v = itrs.velocity.d_xyz.value * 1000.
logger.debug( f'ISS ITRS:\n  - Cartesian: {iss_itrs_p}\n  - Velocity: {iss_itrs_v}\n' )


DEBUG:main:<sgp4.wrapper.Satrec object at 0x13387c610>
DEBUG:main:
  - error code: 0
  - relative_position (km): [-5199.10951535   349.497327    4350.99173184]
  - velocity (km/s): [ 2.47316608 -6.37941418  3.45620405]

DEBUG:main:ISS ITRS:
  - Cartesian: [-1527820.85792497 -4981840.02531517  4350982.71347563]
  - Velocity: [6412.40009404 1062.01783008 3456.20019154]



In [26]:
iss_to_obs = iss_itrs_p - obs_pos_ecef
iss_to_obs_vec = iss_to_obs / np.linalg.norm( iss_to_obs )
print( f'iss_to_obs: {iss_to_obs_vec}' )
print( f'iss_to_obs_vec: {iss_to_obs_vec}' )

iss_to_obs: [-0.57241287 -0.4734443   0.66947292]
iss_to_obs_vec: [-0.57241287 -0.4734443   0.66947292]


In [27]:
iss_tle_pos_lla = ecef_to_lla( iss_itrs_p )
print( f'iss_tle_pos_lla: {iss_tle_pos_lla}' )

iss_tle_pos_lla: (40.0391157528191, -107.04962579308689, 419193.32837435673)


In [28]:
iss_to_obs_ned = ecef_to_ned_matrix( obs_pos_lla ) @ iss_to_obs_vec.reshape((3,1))
print( f'iss_to_obs_ned: {iss_to_obs_ned.flatten()}' )

iss_to_obs_ned: [ 0.13152129 -0.43201888 -0.89222297]


In [29]:
hor = ned_to_horizon( iss_to_obs_ned )
print( f'Horizon. Az: {hor[0]}, El: {hor[1]}' )

Horizon. Az: 286.9320385551661, El: 63.153927743913925
